In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
data = model_data.copy()
# take one copy for model development data
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [5]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(34344, 2832) (10802, 2832)


In [6]:
features_list = [
'num_PERSONAL LOAN_enq_last_24M',
'num_CONSUMER LOAN_enq_last_9M',
'total_emi',
'num_of_loans_le_1lac_non_credit',
'num_CREDIT CARD_enq_last_3M',
'Utilization_all',
'Total_outstanding_Balance_CREDIT CARD_opened_last_6M',
'Age_of_oldest_open_account',
 'Utilization_CREDIT CARD_opened_last_36M',
'highest_sanction_amount_non_credit',
'Total_high_credit',
'num_PL_LE_1_Lac_opened_in_last_3M',
'cash_withdrawal_sum',
'balance_5th_day_M2',
'bounced_transactions_count',
'Min_Balance_Min_M1_M2_M3',
'num_credit_gt_salary',
'upi_trans_count',
'Max_Credit_Avg_M1_M2_M3',
'ratio_total_debit_to_total_credit_M3',

'ratio_6th_day_balance_to_2th_day_balance_M1',

'percentage_ever_delinquent',
'months_since_most_recent_delinquency_all',
'percentage_utilization_gt_75',
'No of enq in past 90days',

'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',

   
'g406s',
'dm206s',
    
'month_year',
'G15_B24']

In [7]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3039234/2552150070.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3039234/2552150070.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20606, 29), (13738, 29), (20606,), (13738,))

In [10]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 29) (4122, 29) (13738, 29)


In [11]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [12]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'lambda': 6,
    'alpha': 4.8,
    'gamma': 5.0,
    'scale_pos_weight': 1,
    'random_state': 42
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 200,
    early_stopping_rounds= 50,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.57807	eval-auc:0.54747
[10]	train-auc:0.63249	eval-auc:0.60259
[20]	train-auc:0.63203	eval-auc:0.60216
[30]	train-auc:0.63355	eval-auc:0.60833
[40]	train-auc:0.63454	eval-auc:0.61013
[50]	train-auc:0.63484	eval-auc:0.61094
[60]	train-auc:0.63508	eval-auc:0.61093
[70]	train-auc:0.63691	eval-auc:0.61735
[80]	train-auc:0.63627	eval-auc:0.61753
[90]	train-auc:0.63725	eval-auc:0.61532
[100]	train-auc:0.63726	eval-auc:0.61493
[110]	train-auc:0.64088	eval-auc:0.61881
[120]	train-auc:0.64236	eval-auc:0.61802
[130]	train-auc:0.64258	eval-auc:0.61988
[140]	train-auc:0.64221	eval-auc:0.61919
[150]	train-auc:0.64292	eval-auc:0.61859
[160]	train-auc:0.64391	eval-auc:0.61814
[170]	train-auc:0.64449	eval-auc:0.61803
[180]	train-auc:0.64564	eval-auc:0.61700
[181]	train-auc:0.64575	eval-auc:0.61648


In [13]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.65, Gini: 0.29, KS: 0.22
Eval AUC: 0.62, Gini: 0.23, KS: 0.18
Test AUC: 0.62, Gini: 0.25, KS: 0.18
OOT AUC: 0.63, Gini: 0.26, KS: 0.19


In [14]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.148575         0.121812        0.159452        0.111535   1649   
9          0.094053         0.101536        0.111518        0.093593   1648   
8          0.082524         0.087786        0.093581        0.082448   1648   
7          0.066101         0.078453        0.082447        0.074733   1649   
6          0.052184         0.071880        0.074732        0.069777   1648   
5          0.055218         0.068710        0.069776        0.067906   1648   
4          0.051578         0.067333        0.067905        0.066832   1648   
3          0.041312         0.066383        0.066831        0.065963   1646   
2          0.036970         0.065590        0.065962        0.065219   1650   
1          0.026061         0.064792        0.065219        0.064134   1650   

        label_good  label_bad  
Decile                         
10            1404        245  
9             1493        155  
8             1512        136  
7             1540        109  
6             1562         86  
5             1557         91  
4             1563         85  
3             1578         68  
2             1589         61  
1             1607         43

In [15]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.120815         0.121829        0.158355        0.111440   1374   
9          0.085881         0.101490        0.111430        0.093439   1374   
8          0.074236         0.087810        0.093429        0.082416   1374   
7          0.067007         0.078468        0.082413        0.074665   1373   
6          0.056041         0.072025        0.074661        0.069853   1374   
5          0.050946         0.068803        0.069852        0.068016   1374   
4          0.055353         0.067431        0.068014        0.066894   1373   
3          0.046579         0.066444        0.066893        0.066009   1374   
2          0.031295         0.065624        0.066009        0.065259   1374   
1          0.024745         0.064821        0.065259        0.064134   1374   

        label_good  label_bad  
Decile                         
10            1208        166  
9             1256        118  
8             1272        102  
7             1281         92  
6             1297         77  
5             1304         70  
4             1297         76  
3             1310         64  
2             1331         43  
1             1340         34

In [16]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.138760         0.122080        0.162300        0.112333   1081   
9          0.118519         0.102779        0.112327        0.094665   1080   
8          0.090741         0.089041        0.094656        0.082848   1080   
7          0.079630         0.078788        0.082843        0.075050   1080   
6          0.072222         0.072350        0.075047        0.070210   1080   
5          0.070370         0.068966        0.070197        0.068114   1080   
4          0.069444         0.067532        0.068113        0.067007   1080   
3          0.054630         0.066539        0.067007        0.066096   1080   
2          0.035382         0.065691        0.066095        0.065349   1074   
1          0.031279         0.064870        0.065348        0.064134   1087   

        label_good  label_bad  
Decile                         
10             931        150  
9              952        128  
8              982         98  
7              994         86  
6             1002         78  
5             1004         76  
4             1005         75  
3             1021         59  
2             1036         38  
1             1053         34

In [17]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
21                                              g406s       103.0
19                          Num_Enq_non_BANK_last_12M        56.0
11                         bounced_transactions_count        48.0
13                                    upi_trans_count        24.0
5   Total_outstanding_Balance_CREDIT CARD_opened_l...        23.0
12                               num_credit_gt_salary        17.0
6                          Age_of_oldest_open_account        16.0
4                                     Utilization_all        15.0
0                      num_PERSONAL LOAN_enq_last_24M        13.0
17                       percentage_utilization_gt_75        12.0
18                           No of enq in past 90days        12.0
1                       num_CONSUMER LOAN_enq_last_9M        11.0
15        ratio_6th_day_balance_to_2th_day_balance_M1        11.0
7             Utilization_CREDIT CARD_opened_last_36M         9.0
9         

In [18]:
#Save Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBI_Train_Pred_1524.csv',index=False)
Eval.to_csv('LeoPayu_BBI_Eval_Pred_1524.csv',index=False)
Test.to_csv('LeoPayu_BBI_Test_Pred_1524.csv',index=False)
Oot.to_csv('LeoPayu_BBI_Oot_Pred_1524.csv',index=False)


In [19]:
#Save the Model
xgb_model.save_model('LeoPayu_BBI_15_in_24mob.json')
print('Model Saved')

Model Saved
